In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

# Hinge loss function
class HingeLoss(nn.Module):
    def __init__(self):
        super(HingeLoss, self).__init__()

    def forward(self, outputs, targets):
        targets = 2 * targets - 1
        hinge_loss = torch.mean(torch.clamp(1 - targets * outputs, min=0))
        return hinge_loss

# Simple linear classifier model
class SimpleLinearModel(nn.Module):
    def __init__(self, input_size):
        super(SimpleLinearModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.linear(x)

# Training function
def train_model_with_loss(model, loss_function, optimizer, X_train_tensor, y_train_tensor, num_epochs=100):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = loss_function(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

# Evaluation function
def evaluate_model(model_bce, model_hinge, X_test_tensor, y_test):
    with torch.no_grad():
        bce_outputs = (torch.sigmoid(model_bce(X_test_tensor)) > 0.5).float()
        hinge_outputs = (model_hinge(X_test_tensor) > 0).float()

    bce_f1 = f1_score(y_test, bce_outputs.cpu().numpy())
    hinge_f1 = f1_score(y_test, hinge_outputs.cpu().numpy())
    return bce_f1, hinge_f1

# Imbalanced dataset generation
def create_imbalanced_data(n_samples=1000, imbalance_ratio=0.1):
    X, y = make_classification(n_samples=n_samples, n_features=2, n_informative=2, n_redundant=0,
                               n_clusters_per_class=1, weights=[imbalance_ratio, 1 - imbalance_ratio], 
                               random_state=42)
    return X, y

# Outlier-heavy dataset generation
def create_outlier_data(n_samples=1000, outlier_ratio=0.05):
    X, y = make_classification(n_samples=int(n_samples * (1 - outlier_ratio)), n_features=2, n_informative=2, n_redundant=0,
                               n_clusters_per_class=1, random_state=42)
    X_outliers = np.random.uniform(-10, 10, size=(int(n_samples * outlier_ratio), 2))
    y_outliers = np.random.randint(0, 2, size=(int(n_samples * outlier_ratio)))
    X = np.vstack((X, X_outliers))
    y = np.hstack((y, y_outliers))
    return X, y

# Hard negatives dataset generation
def create_hard_negative_data(n_samples=1000):
    X, y = make_classification(n_samples=n_samples, n_features=2, n_informative=2, n_redundant=0,
                               n_clusters_per_class=1, class_sep=0.5, random_state=42)
    return X, y

# Main function to run experiments
def run_experiments(n_runs=100):
    imbalanced_bce_f1_scores = []
    imbalanced_hinge_f1_scores = []
    
    outlier_bce_f1_scores = []
    outlier_hinge_f1_scores = []

    hardneg_bce_f1_scores = []
    hardneg_hinge_f1_scores = []

    for i in tqdm(range(n_runs)):
        # 1. Imbalanced Data Experiment
        X, y = create_imbalanced_data()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

        model_bce = SimpleLinearModel(input_size=2)
        criterion_bce = nn.BCEWithLogitsLoss()
        optimizer_bce = optim.SGD(model_bce.parameters(), lr=0.01)
        train_model_with_loss(model_bce, criterion_bce, optimizer_bce, X_train_tensor, y_train_tensor)

        model_hinge = SimpleLinearModel(input_size=2)
        criterion_hinge = HingeLoss()
        optimizer_hinge = optim.SGD(model_hinge.parameters(), lr=0.01)
        train_model_with_loss(model_hinge, criterion_hinge, optimizer_hinge, X_train_tensor, y_train_tensor)

        bce_f1, hinge_f1 = evaluate_model(model_bce, model_hinge, X_test_tensor, y_test)
        imbalanced_bce_f1_scores.append(bce_f1)
        imbalanced_hinge_f1_scores.append(hinge_f1)

        # 2. Outlier Data Experiment
        X, y = create_outlier_data()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

        train_model_with_loss(model_bce, criterion_bce, optimizer_bce, X_train_tensor, y_train_tensor)
        train_model_with_loss(model_hinge, criterion_hinge, optimizer_hinge, X_train_tensor, y_train_tensor)

        bce_f1, hinge_f1 = evaluate_model(model_bce, model_hinge, X_test_tensor, y_test)
        outlier_bce_f1_scores.append(bce_f1)
        outlier_hinge_f1_scores.append(hinge_f1)

        # 3. Hard Negative Data Experiment
        X, y = create_hard_negative_data()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

        train_model_with_loss(model_bce, criterion_bce, optimizer_bce, X_train_tensor, y_train_tensor)
        train_model_with_loss(model_hinge, criterion_hinge, optimizer_hinge, X_train_tensor, y_train_tensor)

        bce_f1, hinge_f1 = evaluate_model(model_bce, model_hinge, X_test_tensor, y_test)
        hardneg_bce_f1_scores.append(bce_f1)
        hardneg_hinge_f1_scores.append(hinge_f1)

    # Calculate averages
    avg_imbalanced_bce_f1 = np.mean(imbalanced_bce_f1_scores)
    avg_imbalanced_hinge_f1 = np.mean(imbalanced_hinge_f1_scores)

    avg_outlier_bce_f1 = np.mean(outlier_bce_f1_scores)
    avg_outlier_hinge_f1 = np.mean(outlier_hinge_f1_scores)

    avg_hardneg_bce_f1 = np.mean(hardneg_bce_f1_scores)
    avg_hardneg_hinge_f1 = np.mean(hardneg_hinge_f1_scores)

    print(f"Average Binary Cross-Entropy F1 Score on Imbalanced Data: {avg_imbalanced_bce_f1:.4f}")
    print(f"Average Hinge Loss F1 Score on Imbalanced Data: {avg_imbalanced_hinge_f1:.4f}")
    print(f"Average Binary Cross-Entropy F1 Score on Outlier Data: {avg_outlier_bce_f1:.4f}")
    print(f"Average Hinge Loss F1 Score on Outlier Data: {avg_outlier_hinge_f1:.4f}")
    print(f"Average Binary Cross-Entropy F1 Score on Hard Negative Data: {avg_hardneg_bce_f1:.4f}")
    print(f"Average Hinge Loss F1 Score on Hard Negative Data: {avg_hardneg_hinge_f1:.4f}")

# Run experiments 100 times
run_experiments(n_runs=1000)

  0%|                                                  | 0/1000 [00:00<?, ?it/s]

> /var/folders/b3/qwjscg712s5fpwffc6h2lt0w0000gn/T/ipykernel_80586/1931995338.py(17)forward()
     15     def forward(self, outputs, targets):
     16         import pdb; pdb.set_trace();
---> 17         targets = 2 * targets - 1
     18         hinge_loss = torch.mean(torch.clamp(1 - targets * outputs, min=0))
     19         return hinge_loss



ipdb>  outputs


tensor([[-1.6436e-01],
        [ 5.6688e-01],
        [ 6.0944e-01],
        [ 2.8114e-01],
        [ 3.8593e-01],
        [ 9.2541e-01],
        [ 1.3400e+00],
        [-5.4667e-01],
        [-4.1929e-01],
        [ 1.9899e-01],
        [-4.6267e-01],
        [ 1.2628e+00],
        [ 4.0881e-01],
        [ 2.7878e-03],
        [-9.2399e-02],
        [ 9.0656e-01],
        [-5.6398e-01],
        [-1.0921e-01],
        [-4.4614e-01],
        [ 1.3335e+00],
        [ 8.4432e-01],
        [ 3.6251e-01],
        [ 9.7574e-01],
        [ 4.1084e-01],
        [ 7.7918e-01],
        [ 1.1612e+00],
        [ 6.0382e-01],
        [ 1.9390e-01],
        [-4.5781e-02],
        [ 9.3577e-01],
        [ 8.0386e-01],
        [ 3.7676e-01],
        [ 1.0894e+00],
        [ 9.8052e-01],
        [ 3.0866e-01],
        [ 6.5430e-01],
        [ 9.1388e-01],
        [ 1.1333e+00],
        [ 2.5813e-01],
        [ 8.5182e-02],
        [-3.4425e-04],
        [ 1.4730e+00],
        [ 4.4124e-01],
        [ 5

ipdb>  type(targets)


...

tensor([[ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 

ipdb>  n


> /var/folders/b3/qwjscg712s5fpwffc6h2lt0w0000gn/T/ipykernel_80586/1931995338.py(19)forward()
     17         targets = 2 * targets - 1
     18         hinge_loss = torch.mean(torch.clamp(1 - targets * outputs, min=0))
---> 19         return hinge_loss
     20 
     21 # Simple linear classifier model



ipdb>  hinge_loss


tensor(0.6498, grad_fn=<MeanBackward0>)
